In [1]:
import pandas as pd
import numpy as np

pd.options.display.max_rows = 1_000

### Pool Particulars

In [2]:
pool_size = 12
starters = {'C': 2, 'LW': 2, 'RW': 2, 'D': 4, 'G': 2}

### Master List

In [3]:
yah = pd.read_csv("../data/yahoo.csv")
cap = pd.read_csv("../data/capfriendly.csv")

# merge
master = pd.merge(yah, cap, how="left", on="name")

# extra features
master["apv"] = master["pick"] 
master["pick"] = master["pick"].rank(method='average', ascending=True)
master['round'] = (master['pick'] // pool_size) + 1

# Organize
master = master[['team', 'age', 'name', 'position', 'apv', 'pick', 'round']]

In [4]:
master.head()

,team,age,name,position,apv,pick,round
0,Edm,23.0,Connor McDavid,C,1.7,1.0,1.0
1,Col,25.0,Nathan MacKinnon,C,2.6,2.0,1.0
2,Edm,25.0,Leon Draisaitl,"C,LW",3.7,3.0,1.0
3,NYR,29.0,Artemi Panarin,LW,5.3,4.0,1.0
4,Was,35.0,Alex Ovechkin,LW,5.6,5.0,1.0


### Multiple Positions

In [5]:
multi = (
    master
    [["name", "position"]]
    .set_index(['name']) 
    .apply(lambda col: col.str.split(',').explode())
    .reset_index()
)

### Projection Data

In [6]:
cbs = pd.read_csv("../data/cbs.csv")
dfo = pd.read_csv("../data/dailyfaceoff.csv")
proj = pd.concat([cbs, dfo])

### Goalies

In [7]:
goalies = proj[proj["position"] == "G"].copy()

GOALIE_CATEGORIES = ["wins", "goals_against_average","saves", "shutouts"]

goalies['goals_against_average'] = -goalies['goals_against_average']

goalies = (
    goalies
    [["name"] + GOALIE_CATEGORIES]
    .groupby("name")
    .mean()
    .apply(lambda x: (x - x.min()) / (x.max() - x.min()))
)

goalies["wins"] *= 1/1
goalies["goals_against_average"] *= 3/4
goalies["saves"] *= 3/4
goalies["shutouts"] *= 1/2

goalies["rollup"] = goalies.apply(lambda row: row.sum(), axis=1)
goalies["rollup"] /= 1 + 3/4 + 3/4 + 1/2
goalies["rollup"] *= 100

goalies = goalies.reset_index()

goalies = goalies[["name", "rollup"]]

### Skaters

In [8]:
skaters = proj[proj["position"] != "G"].copy()

SKATER_CATEGORIES = ['goals', 'assists', 'plus_minus', 'powerplay_points', 'shots_on_goal', 'hits', 'blocks']

skaters = skaters[["name"] + SKATER_CATEGORIES]

skaters = (
    skaters
    [["name"] + SKATER_CATEGORIES]
    .groupby("name")
    .mean()
    .apply(lambda x: (x - x.min()) / (x.max() - x.min()))
)

skaters.sort_values("name")

skaters['goals'] *= 7/8
skaters['assists'] *= 7/8
skaters['plus_minus'] *= 3/4
skaters['powerplay_points'] *= 3/4
skaters['shots_on_goal'] *= 1
skaters['hits'] *= 1
skaters['blocks'] *= 1

skaters["rollup"] = skaters.apply(lambda row: row.sum(), axis=1)
skaters["rollup"] /= 7/8 + 7/8 + 3/4 +3/4 + 1 + 1 + 1
skaters["rollup"] *= 100

skaters = skaters.reset_index()

skaters = skaters[["name", "rollup"]]

### Merge

In [9]:
rollup = pd.concat([skaters, goalies])
rollup = pd.merge(multi, rollup, how="inner", on="name").sort_values("rollup", ascending=False)
rollup.head()

,name,position,rollup
7,Andrei Vasilevskiy,G,93.309038
83,Philipp Grubauer,G,82.063000
35,Tuukka Rask,G,81.849230
39,Robin Lehner,G,81.442042
38,Carter Hart,G,80.446691


### VORP

In [10]:
vorp = rollup.copy()
vorp["vorp"] = vorp["rollup"]

for position, slots in starters.items():
    replacement = (
        vorp[vorp['position'] == position]
        .sort_values('vorp', ascending=False)
        .head(slots * pool_size)
        ['vorp']
        .mean()
    )
    vorp.loc[vorp['position'] == position, 'vorp'] = vorp['vorp'] - replacement

vorp.head()

,name,position,rollup,vorp
7,Andrei Vasilevskiy,G,93.309038,20.816607
83,Philipp Grubauer,G,82.063000,9.570569
35,Tuukka Rask,G,81.849230,9.356799
39,Robin Lehner,G,81.442042,8.949611
38,Carter Hart,G,80.446691,7.954260


### Value Over Next Round

In [11]:
vnr = pd.merge(vorp, master[["name", "round"]], how="inner", on="name")

vnr = (
    vnr
    .sort_values(["rollup"], ascending=[False])
    .groupby(["round", "position"])
    .head(1)
    .sort_values(["position", "round"])
)

vnr["round"] = vnr["round"] - 1
vnr["next"] = vnr["rollup"]
vnr = vnr[["round", "position", "next"]]

### Draft

In [12]:
draft = pd.merge(master, vorp, how="left", on="name", suffixes=("_yahoo", ""))
draft = pd.merge(draft, vnr, how="left", on=["position", "round"])

draft = draft.sort_values("vorp", ascending=False)
draft = draft.groupby("name").head(1)

draft["vorn"] = draft["rollup"] - draft["next"]
draft['rank'] = draft['vorp'].rank(method='average', ascending=False)
draft["arbitrage"] = draft["pick"] - draft["rank"]

draft = draft[[
    'team', 'age', 'name', 'position_yahoo', 
    'rollup', 'vorp', 'vorn',
    'round', 'pick', 'rank', 'arbitrage'
]]

draft["rollup"] = draft["rollup"].round(1)
draft["vorp"] = draft["vorp"].round(1)
draft["vorn"] = draft["vorn"].round(1)

draft.head()

,team,age,name,position_yahoo,rollup,vorp,vorn,round,pick,rank,arbitrage
7,TB,26.0,Andrei Vasilevskiy,G,93.3,20.8,15.1,1.0,7.0,1.0,6.0
1,Col,25.0,Nathan MacKinnon,C,63.0,15.9,5.6,1.0,2.0,2.0,0.0
34,Nsh,30.0,Roman Josi,D,53.2,13.1,3.8,3.0,30.0,3.0,27.0
0,Edm,23.0,Connor McDavid,C,59.0,11.9,1.5,1.0,1.0,4.0,-3.0
5,Was,35.0,Alex Ovechkin,LW,57.2,11.6,5.9,1.0,5.0,5.0,0.0


### Targets

In [13]:
draft["target"] = (
    (draft["age"] <= 30) &
    (draft["vorp"] >= -1) &
    (draft["arbitrage"] >= -12) & 
    (draft["vorn"] >= 0)
)

In [14]:
draft.to_csv("../data/draft.csv", index=False)